# SMA Crossover Strategy with Martingale
Author: Mohamed Abbas El kayal

## Disclaimer
Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

In [1117]:
import datetime
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np

from datetime import datetime
from IPython.display import display

In [1118]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [1119]:
# connect to MetaTrader5 as mt5
mt5.initialize()

True

In [1120]:
# settings
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_H1
start_pos = 0
num_bars = 9500

fast_sma_period = 5
slow_sma_period = 200

In [1121]:
# Requesting historical data from MetaTrader 5
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)

# Converting bars we got from MetaTrader5 into dataframe
df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close', 'spread']]

#  Re-factoring time format into human readable
df['time'] = pd.to_datetime(df['time'], unit='s')


In [1122]:
# Visualize Close Price
px.line(df, x='time', y='close')

In [1123]:
#  define the indicators
def indicator_sma (dataframe):
    dataframe['fast_sma'] = dataframe['close'].rolling(fast_sma_period).mean()
    dataframe['slow_sma'] = dataframe['close'].rolling(slow_sma_period).mean()

    # finding crossovers
    dataframe['prev_fast_sma'] = dataframe['fast_sma'].shift(1)

    dataframe.dropna(inplace=True)

indicator_sma(dataframe=df)

In [1124]:
def create_signal(dataframe):
    
    conditions_signal = [
        # dataframe['fast_sma'].gt(dataframe['slow_sma']) & dataframe['prev_fast_sma'].lt(dataframe['slow_sma']), # Buy Signal
        # dataframe['fast_sma'].lt(dataframe['slow_sma']) & dataframe['prev_fast_sma'].gt(dataframe['slow_sma']), # Sell Signal
        dataframe['fast_sma'].gt(dataframe['slow_sma']), # up signal
        dataframe['fast_sma'].lt(dataframe['slow_sma']), # down signal
    ]

    choices_signal = [
        # 'buy',
        # 'sell', 
        'buy', 
        'sell'
        ]

    dataframe['signal'] = np.select(conditions_signal, choices_signal, default=None)

    # export dataframe to excel file

    # create file name based on current date and time
    # filename = datetime.now().strftime('%Y-%m-%d-%H-%M-%S.xlsx')

    # export dataframe to excel file
    # df.to_excel(filename)  

    dataframe['signal'] = dataframe['signal'].shift(1)
    
    return dataframe

create_signal(dataframe=df)

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma,signal
199,2022-07-28 17:00:00,1.01388,1.01676,1.01380,1.01553,19,1.014134,1.018885,1.014078,NaN
200,2022-07-28 18:00:00,1.01553,1.01703,1.01451,1.01645,19,1.014766,1.018894,1.014134,sell
201,2022-07-28 19:00:00,1.01644,1.01644,1.01411,1.01570,19,1.015562,1.018908,1.014766,sell
202,2022-07-28 20:00:00,1.01570,1.01658,1.01543,1.01590,19,1.015488,1.018910,1.015562,sell
203,2022-07-28 21:00:00,1.01590,1.01760,1.01552,1.01758,19,1.016232,1.018920,1.015488,sell
...,...,...,...,...,...,...,...,...,...,...
9495,2024-01-26 19:00:00,1.08639,1.08695,1.08583,1.08626,17,1.086672,1.087484,1.086750,sell
9496,2024-01-26 20:00:00,1.08625,1.08650,1.08529,1.08549,18,1.086348,1.087471,1.086672,sell
9497,2024-01-26 21:00:00,1.08548,1.08608,1.08544,1.08573,16,1.086036,1.087454,1.086348,sell
9498,2024-01-26 22:00:00,1.08575,1.08613,1.08521,1.08534,16,1.085844,1.087438,1.086036,sell


In [1125]:
def create_ohlcv(dataframe):
  deals = dataframe.groupby((dataframe.signal != dataframe.signal.shift()).cumsum(), as_index= False).agg(
    open_time = ('time', 'first'),
    signal = ('signal', 'first'),
    open_price = ('open', 'first'),
    highest = ('high', 'max'),
    lowest = ('low', 'min')
  )

  safety = 1
  deals['open_price'] = round(deals['open_price'] * safety, 5)
  deals['close_price'] = deals['open_price'].shift(-1)
  deals['close_price'].fillna(value=dataframe['close'].iloc[-2], inplace=True)
  
  deals.dropna(inplace=True)
  
  return deals

deals = create_ohlcv(dataframe=df)

deals

,open_time,signal,open_price,highest,lowest,close_price
1,2022-07-28 18:00:00,sell,1.01553,1.01987,1.01411,1.01911
2,2022-07-29 03:00:00,buy,1.01911,1.02532,1.01451,1.02005
3,2022-07-29 19:00:00,sell,1.02005,1.02196,1.01763,1.02190
4,2022-07-29 22:00:00,buy,1.02190,1.02926,1.01701,1.01830
5,2022-08-02 21:00:00,sell,1.01830,1.02329,1.01219,1.02212
...,...,...,...,...,...,...
143,2024-01-16 03:00:00,sell,1.09189,1.09307,1.08207,1.09237
144,2024-01-24 16:00:00,buy,1.09237,1.09314,1.08783,1.08788
145,2024-01-24 23:00:00,sell,1.08788,1.08987,1.08692,1.08923
146,2024-01-25 13:00:00,buy,1.08923,1.09006,1.08519,1.08594


In [1126]:
def calc_profit_dots(row):
    if row['signal'] == 'buy':
        val = row['close_price'] - row['open_price']
    elif row['signal'] == 'sell':
        val = row['open_price'] - row['close_price']
    else:
        val = 0
    return val

def calc_broker_digits(broker_digits):
    if broker_digits == 4:
        digits_factor = 1000
    elif broker_digits == 5:
        digits_factor = 10000
    return digits_factor

def apply_profit_dots(dataframe, function):
    dataframe['dots'] = dataframe.apply(function, axis=1)
    dataframe['points'] = dataframe['dots'] * calc_broker_digits(broker_digits=5)
    dataframe['type'] = np.where(dataframe['dots'] > 0, 'win', 'loss')
    # dataframe['mart_factor'] = (dataframe['profit_type'].groupby((dataframe['profit_type'] != dataframe['profit_type'].shift()).cumsum()).cumcount() + 1)
    dataframe.reset_index(drop=True, inplace=True)
    
    return dataframe

deals = apply_profit_dots(dataframe=deals, function=calc_profit_dots)

deals

,open_time,signal,open_price,highest,lowest,close_price,dots,points,type
0,2022-07-28 18:00:00,sell,1.01553,1.01987,1.01411,1.01911,-0.00358,-35.8,loss
1,2022-07-29 03:00:00,buy,1.01911,1.02532,1.01451,1.02005,0.00094,9.4,win
2,2022-07-29 19:00:00,sell,1.02005,1.02196,1.01763,1.02190,-0.00185,-18.5,loss
3,2022-07-29 22:00:00,buy,1.02190,1.02926,1.01701,1.01830,-0.00360,-36.0,loss
4,2022-08-02 21:00:00,sell,1.01830,1.02329,1.01219,1.02212,-0.00382,-38.2,loss
...,...,...,...,...,...,...,...,...,...
142,2024-01-16 03:00:00,sell,1.09189,1.09307,1.08207,1.09237,-0.00048,-4.8,loss
143,2024-01-24 16:00:00,buy,1.09237,1.09314,1.08783,1.08788,-0.00449,-44.9,loss
144,2024-01-24 23:00:00,sell,1.08788,1.08987,1.08692,1.08923,-0.00135,-13.5,loss
145,2024-01-25 13:00:00,buy,1.08923,1.09006,1.08519,1.08594,-0.00329,-32.9,loss


In [1127]:
def calc_martingale_factor(dataframe):
    
    dataframe['profit_code'] = np.where(dataframe['dots'] > 0, 0, 1)

    profit_code = list(deals['profit_code'])

    profit_code = pd.Series(profit_code)

    steps = (profit_code.cumsum() * profit_code).diff() # mask out the cumsum where we won [0 1 2 3 0 0 4 5 6 ... ]

    edges = steps < 0 # find where the cumsum steps down -> where we won
    diff_steps = steps[edges].diff() # find the length of each losing streak
    diff_steps[steps[edges].index[0]] = steps[edges][:1] # fix length of the first run which in now NaN
    toss = profit_code.copy() # get a copy of the profit_code series
    toss[edges] = diff_steps # insert the length of the losing streaks into the copy of the profit_code results
    martingale_factor = 2 ** (toss).cumsum() # compute the wagers

    result = pd.DataFrame({
        'profit_code': profit_code,
        'toss': toss,
        'runs': toss.cumsum(),
        'martingale': martingale_factor
        })
    
    result['profit_toss'] = np.where(result['profit_code'] == 0, 'win', 'loss')
    
    result['martingale'] = result['martingale'].shift(periods=1, fill_value=1)
    
    # drop profit_code from dataframe
    dataframe = dataframe.drop('profit_code', axis=1) 
    
    # add mart_profit to the dataframe
    dataframe['martingale']= result['martingale']
    return dataframe

deals = calc_martingale_factor(dataframe=deals)

deals

,open_time,signal,open_price,highest,lowest,close_price,dots,points,type,martingale
0,2022-07-28 18:00:00,sell,1.01553,1.01987,1.01411,1.01911,-0.00358,-35.8,loss,1
1,2022-07-29 03:00:00,buy,1.01911,1.02532,1.01451,1.02005,0.00094,9.4,win,2
2,2022-07-29 19:00:00,sell,1.02005,1.02196,1.01763,1.02190,-0.00185,-18.5,loss,1
3,2022-07-29 22:00:00,buy,1.02190,1.02926,1.01701,1.01830,-0.00360,-36.0,loss,2
4,2022-08-02 21:00:00,sell,1.01830,1.02329,1.01219,1.02212,-0.00382,-38.2,loss,4
...,...,...,...,...,...,...,...,...,...,...
142,2024-01-16 03:00:00,sell,1.09189,1.09307,1.08207,1.09237,-0.00048,-4.8,loss,8
143,2024-01-24 16:00:00,buy,1.09237,1.09314,1.08783,1.08788,-0.00449,-44.9,loss,16
144,2024-01-24 23:00:00,sell,1.08788,1.08987,1.08692,1.08923,-0.00135,-13.5,loss,32
145,2024-01-25 13:00:00,buy,1.08923,1.09006,1.08519,1.08594,-0.00329,-32.9,loss,64


In [1128]:
# Lot Size:
# Standard => Volume = 100,000 => Lot = 1.00 => pip Value = 10.00 USD
# Mini Lot => Volume = 10,000 => Lot = 0.10 => pip Value = 1.00 USD
# Micro Lot => Volume = 1,000 => Lot = 0.01 => pip Value = 0.10 USD
# Nano Lot => Volume = 100 => Lot = 0.0001 => pip Value = 10 USD

deposit = 1000
contract = 10000000
min_lot= 0.01
volume = contract * min_lot


def create_profit_values(dataframe):
    dataframe['lot'] = dataframe['martingale'] * min_lot
    dataframe['profit'] = dataframe['dots'] * dataframe['lot'] * volume
    dataframe['pnl_points'] = dataframe['points'].cumsum()
    dataframe['pnl_value'] = dataframe['profit'].cumsum() + deposit
    # dataframe['fast_sma_period'] = fast_sma_period
    # dataframe['slow_sma_period'] = slow_sma_period
    return dataframe

deals = create_profit_values(dataframe=deals)

deals

,open_time,signal,open_price,highest,lowest,close_price,dots,points,type,martingale,lot,profit,pnl_points,pnl_value
0,2022-07-28 18:00:00,sell,1.01553,1.01987,1.01411,1.01911,-0.00358,-35.8,loss,1,0.01,-3.58,-35.8,996.42
1,2022-07-29 03:00:00,buy,1.01911,1.02532,1.01451,1.02005,0.00094,9.4,win,2,0.02,1.88,-26.4,998.30
2,2022-07-29 19:00:00,sell,1.02005,1.02196,1.01763,1.02190,-0.00185,-18.5,loss,1,0.01,-1.85,-44.9,996.45
3,2022-07-29 22:00:00,buy,1.02190,1.02926,1.01701,1.01830,-0.00360,-36.0,loss,2,0.02,-7.20,-80.9,989.25
4,2022-08-02 21:00:00,sell,1.01830,1.02329,1.01219,1.02212,-0.00382,-38.2,loss,4,0.04,-15.28,-119.1,973.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,2024-01-16 03:00:00,sell,1.09189,1.09307,1.08207,1.09237,-0.00048,-4.8,loss,8,0.08,-3.84,-1133.4,191655.43
143,2024-01-24 16:00:00,buy,1.09237,1.09314,1.08783,1.08788,-0.00449,-44.9,loss,16,0.16,-71.84,-1178.3,191583.59
144,2024-01-24 23:00:00,sell,1.08788,1.08987,1.08692,1.08923,-0.00135,-13.5,loss,32,0.32,-43.20,-1191.8,191540.39
145,2024-01-25 13:00:00,buy,1.08923,1.09006,1.08519,1.08594,-0.00329,-32.9,loss,64,0.64,-210.56,-1224.7,191329.83


In [1129]:
def calc_max_tp_sl(dataframe):
    dataframe['max_tp_points'] = np.where(dataframe['signal'] == 'buy', (dataframe['highest'] - dataframe['open_price']) * calc_broker_digits(broker_digits=5), 
                                   (dataframe['open_price'] - dataframe['lowest']) * calc_broker_digits(broker_digits=5))
    dataframe['max_sl_points'] = np.where(dataframe['signal'] == 'buy', (dataframe['open_price'] - dataframe['lowest'])  * calc_broker_digits(broker_digits=5), 
                                   (dataframe['highest'] - dataframe['open_price']) * calc_broker_digits(broker_digits=5))
    return dataframe

deals = calc_max_tp_sl(dataframe=deals)

pd.set_option('display.max_rows', None)
deals

,open_time,signal,open_price,highest,lowest,close_price,dots,points,type,martingale,lot,profit,pnl_points,pnl_value,max_tp_points,max_sl_points
0,2022-07-28 18:00:00,sell,1.01553,1.01987,1.01411,1.01911,-0.00358,-35.8,loss,1,0.01,-3.58,-35.8,996.42,14.2,43.4
1,2022-07-29 03:00:00,buy,1.01911,1.02532,1.01451,1.02005,0.00094,9.4,win,2,0.02,1.88,-26.4,998.30,62.1,46.0
2,2022-07-29 19:00:00,sell,1.02005,1.02196,1.01763,1.02190,-0.00185,-18.5,loss,1,0.01,-1.85,-44.9,996.45,24.2,19.1
3,2022-07-29 22:00:00,buy,1.02190,1.02926,1.01701,1.01830,-0.00360,-36.0,loss,2,0.02,-7.20,-80.9,989.25,73.6,48.9
4,2022-08-02 21:00:00,sell,1.01830,1.02329,1.01219,1.02212,-0.00382,-38.2,loss,4,0.04,-15.28,-119.1,973.97,61.1,49.9
5,2022-08-04 19:00:00,buy,1.02212,1.02529,1.01406,1.01657,-0.00555,-55.5,loss,8,0.08,-44.40,-174.6,929.57,31.7,80.6
6,2022-08-05 18:00:00,sell,1.01657,1.02185,1.01504,1.02161,-0.00504,-50.4,loss,16,0.16,-80.64,-225.0,848.93,15.3,52.8
7,2022-08-08 18:00:00,buy,1.02161,1.02209,1.01843,1.01913,-0.00248,-24.8,loss,32,0.32,-79.36,-249.8,769.57,4.8,31.8
8,2022-08-08 23:00:00,sell,1.01913,1.02437,1.01875,1.02384,-0.00471,-47.1,loss,64,0.64,-301.44,-296.9,468.13,3.8,52.4
9,2022-08-09 12:00:00,buy,1.02384,1.03678,1.01859,1.01984,-0.00400,-40.0,loss,128,1.28,-512.00,-336.9,-43.87,129.4,52.5


In [1130]:
def calc_result(dataframe):
    
    
    
    # find balance
    balance = round(dataframe['pnl_value'].iloc[-1], 2)
    print('balance = ', balance)

    # find total net profit
    total_net_profit = round(dataframe['profit'].sum(), 2)  # Use in backtest Factors
    print('total net profit = ', total_net_profit)

    # find gross profit
    gross_profit = round(sum(dataframe[dataframe['type']=='win']['profit']), 2)
    print('gross profit = ', gross_profit)

    # find gross loss
    gross_loss = round(sum(dataframe[dataframe['type']=='loss']['profit']), 2)
    print('gross loss = ', gross_loss)

    # find profit factor
    profit_factor = abs(round((gross_profit / gross_loss), 2))
    print('profit factor = ', profit_factor)
    
    # find largest profit trade
    largest_profit_trade = round(dataframe['profit'].max(), 2)
    print('largest profit trade = ', largest_profit_trade)
    
    # find largest profit trade
    df2=dataframe.loc[dataframe['profit'] == dataframe['profit'].max(), 'open_time'].item()
    print('largest profit trade open time = ', df2)

    # find largest loss trade
    largest_loss_trade = round(dataframe['profit'].min(), 2)
    print('largest loss trade = ', largest_loss_trade)

    # find total trades
    total_trades = len(dataframe)
    print('total trades = ', total_trades)

    # find total count of SELL trades
    count_sell_trades = dataframe['signal'].value_counts()['sell']
    print('count SELL trades = ', count_sell_trades)

    # find total count of BUY trades
    count_buy_trades = dataframe['signal'].value_counts()['buy']
    print('count BUY trades = ', count_buy_trades)

    # find total count of WIN trades
    count_win_trades = dataframe['type'].value_counts()['win']
    print('count WIN trades = ', count_win_trades)
    
    # find total count of LOSS trades
    count_loss_trades = dataframe['type'].value_counts()['loss']
    print('count LOSS trades = ', count_loss_trades)

    # select rows by value
    # avg_win = deals_group.loc[deals_group['profit_type'] == 'win']
    # print('avg_win' , avg_win)

    # find average value of WIN Trades
    ave_win_value = round(dataframe.loc[dataframe['type'] == 'win']['profit'].mean() , 2)
    print('ave_win_value = ' , ave_win_value)

    # find average value of LOSS Trades
    ave_loss_value = round(dataframe.loc[dataframe['type'] == 'loss']['profit'].mean(), 2)
    print('ave_loss_value = ' , ave_loss_value)
    
    # find max take profit points
    max_tp_points = round(dataframe['max_tp_points'].max(), 2)
    print('max take profit points = ', max_tp_points)
 
    # find average take profit points
    avg_tp_points = round(dataframe['max_tp_points'].mean(), 2)
    print('average take profit points = ', avg_tp_points)   
    
    # find min take profit points
    min_tp_points = round(dataframe['max_tp_points'].min(), 2)
    print('min take profit points = ', min_tp_points)   
    
    # find max stop loss points
    max_sl_points = round(dataframe['max_sl_points'].max(), 2)
    print('max stop loss points = ', max_sl_points)
    
    # find average stop loss points
    avg_sl_points = round(dataframe['max_sl_points'].mean(), 2)
    print('average stop loss points = ', avg_sl_points) 
    
    # find min stop loss points
    min_sl_points = round(dataframe['max_sl_points'].min(), 2)
    print('min stop loss points = ', min_sl_points)
    
    # find max take profit points open time
    df3=dataframe.loc[dataframe['max_tp_points'] == dataframe['max_tp_points'].max(), 'open_time'].item()
    print('max take profit points open time = ', df3)
    
    # find max stop loss points open time
    df4=dataframe.loc[dataframe['max_sl_points'] == dataframe['max_sl_points'].max(), 'open_time'].item()
    print('max stop loss points open time = ', df4)
    
    # Collect consecutive Profit / Loss (Factors)

    # find consecutive Profit / Loss
    positions_group = dataframe.groupby((dataframe.type != dataframe.type.shift()).cumsum()).agg(
    type = ('type', 'first'),
    grand_profit = ('profit', 'sum'),
    grand_points = ('points', 'sum'),
    count = ('type', 'count')
    ).reset_index(drop=True)

    print('count trades groups = ')
    print(positions_group)

    # find largest consecutive WIN trade
    largest_cons_win = round(positions_group['grand_profit'].max(), 2)
    print('largest consecutive WIN trade = ', largest_cons_win)

    # find largest consecutive WIN trade (count)
    count_largest_win = positions_group.loc[positions_group['grand_profit'].idxmax(), 'count']
    print('largest consecutive WIN trade (count) = ', count_largest_win)

    # find largest consecutive LOSS trade
    largest_cons_loss = round(positions_group['grand_profit'].min(), 2)
    print('largest consecutive LOSS trade = ', largest_cons_loss)

    # find largest consecutive LOSS trade (count)
    count_largest_loss = positions_group.loc[positions_group['grand_profit'].idxmin(), 'count']
    print('largest consecutive LOSS trade (count) = ', count_largest_loss)
    
    result = {
        'fast_sma': [fast_sma_period], # fast SMA
        'slow_sma': [slow_sma_period], # slow SMA
        'deposit': [deposit], # deposit
        'balance': [balance], # balance
        'pnl': [total_net_profit],
        'gross_profit': [gross_profit],
        'gross_loss': [gross_loss],
        'profit_factor': [profit_factor],
        'max_win': [largest_profit_trade],
        'max_loss': [largest_loss_trade],
        'total_trades': [total_trades],
        'count_sell': [count_sell_trades],
        'count_buy': [count_buy_trades],
        'count_win': [count_win_trades],
        'count_loss': [count_loss_trades],
        'ave_win': [ave_win_value],
        'ave_loss': [ave_loss_value],
        'max_tp_points': [max_tp_points],
        'avg_tp_points': [avg_tp_points],
        'min_tp_points': [min_tp_points],
        'max_sl_points': [max_sl_points],
        'avg_sl_points': [avg_sl_points],
        'min_sl_points': [min_sl_points],
        'max_cons_win': [largest_cons_win],
        'count_max_cons_win': [count_largest_win],
        'max_cons_loss': [largest_cons_loss],
        'count_max_cons_loss': [count_largest_loss],
    }
    result_df = pd.DataFrame.from_dict(result)
    return result_df

result_df = calc_result(dataframe=deals)

result_df

balance =  191406.63
total net profit =  190406.63
gross profit =  215414.81
gross loss =  -25008.18
profit factor =  8.61
largest profit trade =  184975.36
largest profit trade open time =  2022-09-20 09:00:00
largest loss trade =  -5550.08
total trades =  147
count SELL trades =  74
count BUY trades =  73
count WIN trades =  29
count LOSS trades =  118
ave_win_value =  7428.1
ave_loss_value =  -211.93
max take profit points =  533.3
average take profit points =  70.33
min take profit points =  0.0
max stop loss points =  118.4
average stop loss points =  39.35
min stop loss points =  0.5
max take profit points open time =  2022-11-04 21:00:00
max stop loss points open time =  2022-08-30 11:00:00
count trades groups = 
    type  grand_profit  grand_points  count
0   loss         -3.58         -35.8      1
1    win          1.88           9.4      1
2   loss      -1042.17        -310.5      8
3    win       4352.00         170.0      1
4   loss     -14951.95        -423.7     13
5    w

,fast_sma,slow_sma,deposit,balance,pnl,gross_profit,gross_loss,profit_factor,max_win,max_loss,...,max_tp_points,avg_tp_points,min_tp_points,max_sl_points,avg_sl_points,min_sl_points,max_cons_win,count_max_cons_win,max_cons_loss,count_max_cons_loss
0,5,200,1000,191406.63,190406.63,215414.81,-25008.18,8.61,184975.36,-5550.08,...,533.3,70.33,0.0,118.4,39.35,0.5,184975.36,1,-14951.95,13


In [1131]:
px.line(deals, x='open_time', y='pnl_value')

In [1132]:
# def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
#     if fast_sma > slow_sma and prev_fast_sma < slow_sma:
#         return 'bullish crossover'
#     elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
#         return 'bearish crossover'
    
#     return None


# df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

# # df.dropna(inplace=True)
# # signal = df[df['crossover'] == 'bullish crossover'].copy() # => Original
# signal = df[(df['crossover'] == 'bullish crossover') | (df['crossover'] == 'bearish crossover')].copy()

# signal

In [1133]:
# # visualize close price
# fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

# for i, row in signal.iterrows():
#     fig.add_vline(x=row.time)
    
# fig.show()

In [1134]:
# # visualize close price
# fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

# for i, row in signal.iterrows():
#     fig.add_vline(x=row.time)
    
# for i, row in deals[deals['status'] == 'closed'].iterrows():
    
#     if row.profit > 0:
#         fig.add_shape(type="line",
#             x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
#             line=dict(color="Green",width=3)
#                      )
                      
#     elif row.profit < 0:
#         fig.add_shape(type="line",
#             x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
#             line=dict(color="Red",width=3)
#                       )

    
# fig.show()

In [1135]:
# Create Summary DataFrame
summary = pd.DataFrame(columns=['fast_sma',
                                'slow_sma',
                                
                                'deposit',  # deposit
                                'balance',  # balance
                                
                                'net_profit', # total net profit
                                
                                'gross_profit', # sum of win trades
                                'gross_loss', # sum of fail trades
                                'profit_factor', # ratio (%) => (gross profit / gross loss) => one means (profit = loss)
                                
                                'abs_drawdown', # the largest loss is lower than the initial deposit value
                                'max_drawdown', # maximal loss of the local maximum in the deposit currency and in percents of the deposit
                                'rel_drawdown', #  the maximal loss in percents of the maximum equity value and its corresponding money value
                                
                                'total_trades', # count total trades
                                'total_buy', # count total BUY trades
                                'total_sell', # count total SELL trades
                                
                                'max_win', # Largest profit trade
                                'max_fail', # Largest loss trade
                                
                                'min_lot', # initial lot
                                'lot_multiplier',
                                'max_lot', # highest lot
                                
                                'F',
                                'G'])

# balance =  9971.76
# total net profit =  -28.24
# gross profit =  5.43
# gross loss =  -33.67
# profit factor =  0.16
# largest profit trade =  5.2
# largest loss trade =  -2.2
# total trades =  65
# count SELL trades =  24
# count BUY trades =  41
# count WIN trades =  5
# count LOSS trades =  60
# ave_win_value =  1.09
# ave_loss_value =  -0.56

summary

,fast_sma,slow_sma,deposit,balance,net_profit,gross_profit,gross_loss,profit_factor,abs_drawdown,max_drawdown,...,total_trades,total_buy,total_sell,max_win,max_fail,min_lot,lot_multiplier,max_lot,F,G
